In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import math
from math import sqrt
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
# import SeabornFig2Grid as sfg
from matplotlib.pylab import rcParams
import matplotlib.gridspec as gridspec
from scipy.stats import zscore
import csv
import codecs
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%config InlineBackend.figure_format = 'retina'

In [3]:
data = []

with codecs.open('dataset/good.csv', encoding='utf-8', errors='ignore') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        data.append(row)
        
data1 = pd.DataFrame(data)
data1.shape

(4559, 99)

In [4]:
data = []

with codecs.open('dataset/bad.csv', encoding='utf-8', errors='ignore') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        data.append(row)
        
data2 = pd.DataFrame(data)
data2.shape

(135, 99)

In [5]:
# 留下的特徵
all_name = [
#     'ShotSN',
#     'MoldClampedTime',
#     'MoldReleasedTime',
#     'MOID',
#     'ProdID',
    'Sensor2_MaxPressure',
    'Sensor2_SumStage1Pressure',
#     'Sensor2_MaxPressureTime',
    'Sensor2_AvgStage1PressureSlope',
    'Sensor2_AvgStage2PressureSlope',
    'Sensor2_MoldReleasePressure',
    'Sensor3_MaxPressure',
    'Sensor3_SumStage1Pressure',
#     'Sensor3_MaxPressureTime',
    'Sensor3_AvgStage1PressureSlope',
    'Sensor3_AvgStage2PressureSlope',
    'Sensor3_MoldReleasePressure',
    'Sensor4_MaxPressure',
    'Sensor4_SumStage1Pressure',
#     'Sensor4_MaxPressureTime',
    'Sensor4_AvgStage1PressureSlope',
    'Sensor4_AvgStage2PressureSlope',
    'Sensor4_MoldReleasePressure',
    'Sensor5_StartTemperature',
    'Sensor5_MaxTemperature',
    'Sensor5_AvgStage2TemperatureSlope',
    'Sensor5_MoldReleaseTemperature',
#     'Sensor1_Stage1StartTime',
#     'Sensor1_Stage1EndTime',
#     'Sensor1_Stage1Period',
    'Sensor1_AvgStage1PressureSlope',
    'Sensor1_SumStage1Pressure',
    'Sensor1_Stage1EndPressure',
#     'Sensor1_Stage2StartTime',
#     'Sensor1_Stage2EndTime',
#     'Sensor1_Stage2Period',
    'Sensor1_AvgStage2PressureSlope',
    'Sensor1_SumStage2Pressure',
#     'Sensor1_Stage3StartTime',
#     'Sensor1_Stage3EndTime',
#     'Sensor1_Stage3Period',
    'Sensor1_MaxStage3Pressure',
#     'Sensor1_MaxStage3PressureTime',
    'Sensor1_SumStage3Pressure',
#     'Sensor1_CycleTime',
    'Sensor1_SumPressure',
#     'Soft_MoldClampedCycleTime',
#     'Soft_MoldBasedCycleTime',
    'Soft_MoldShotInterval',
#     'Soft_MoldCompleteCycleTime',
    'Soft_DiffMaxPressure',
#     'Soft_DiffIncreasePressureTime',
    'Soft_DiffMoldReleasePressure',
    'Type',
]

In [6]:
data1.columns = data1.iloc[0]
data1 = data1.iloc[2:, :]
data1 = data1.reset_index(drop=True)
data1.head()

,ShotSN,MoldClampedTime,MoldReleasedTime,MOID,ProdID,Sensor2_MaxPressure,Sensor2_SumStage1Pressure,Sensor2_MaxPressureTime,Sensor2_AvgStage1PressureSlope,Sensor2_AvgStage2PressureSlope,Sensor2_MoldReleasePressure,Sensor3_MaxPressure,Sensor3_SumStage1Pressure,Sensor3_MaxPressureTime,Sensor3_AvgStage1PressureSlope,Sensor3_AvgStage2PressureSlope,Sensor3_MoldReleasePressure,Sensor4_MaxPressure,Sensor4_SumStage1Pressure,Sensor4_MaxPressureTime,Sensor4_AvgStage1PressureSlope,Sensor4_AvgStage2PressureSlope,Sensor4_MoldReleasePressure,Sensor5_StartTemperature,Sensor5_MaxTemperature,Sensor5_AvgStage2TemperatureSlope,Sensor5_MoldReleaseTemperature,Sensor1_Stage1StartTime,Sensor1_Stage1EndTime,Sensor1_Stage1Period,Sensor1_AvgStage1PressureSlope,Sensor1_SumStage1Pressure,Sensor1_Stage1EndPressure,Sensor1_Stage2StartTime,Sensor1_Stage2EndTime,Sensor1_Stage2Period,Sensor1_AvgStage2PressureSlope,Sensor1_SumStage2Pressure,Sensor1_Stage3StartTime,Sensor1_Stage3EndTime,Sensor1_Stage3Period,Sensor1_MaxStage3Pressure,Sensor1_MaxStage3PressureTime,Sensor1_SumStage3Pressure,Sensor1_CycleTime,Sensor1_SumPressure,Soft_MoldClampedCycleTime,Soft_MoldBasedCycleTime,Soft_MoldShotInterval,Soft_MoldCompleteCycleTime,Soft_DiffMaxPressure,Soft_DiffIncreasePressureTime,Soft_DiffMoldReleasePressure,OPCUA_CycleTime,OPCUA_MoldClampingTime,OPCUA_MoldReleasingTime,OPCUA_InjectionTime,OPCUA_VPChangeOverDelayTime,OPCUA_HoldingTime,OPCUA_PlastificationTime,OPCUA_VPChangeOverPosition,OPCUA_CusionVolume,OPCUA_EndHoldingPosition,OPCUA_EndPlastificationPosition,OPCUA_Nozzle1Temperature,OPCUA_Nozzle2Temperature,OPCUA_Nozzle3Temperature,OPCUA_Nozzle4Temperature,OPCUA_Nozzle5Temperature,OPCUA_IMMOilTemperature,OPCUA_FallMaterialTemperature,OPCUA_MaxInjectionPressure,OPCUA_VPChangeOverPressure,OPCUA_SumInjectionPressure,OPCUA_SumHoldingPressure,OPCUA_MaxCavityNearGatePressure,OPCUA_MaxCavityFarGatePressure,OPCUA_SumCavityPressure,OPCUA_SumMoldClampingForce,OPCUA_MaxMoldClampingForce,OPCUA_MaxBackPressure,OPCUA_DelayPlastificationTime,OPCUA_InMoldCoolingTemperature,OPCUA_OutMoldCoolingTemperature,OPCUA_MoldCoolingVolume,OPCUA_MoldReleasingPosition2,OPCUA_MoldReleasingPosition3,OPCUA_MoldReleasingPosition4,OPCUA_MoldReleasingPosition5,OPCUA_ScrewSuckBackPosition,OPCUA_ScrewRPM,OPCUA_PartWeight,OPCUA_GateClosingTime,OPCUA_MeltTemperature,OPCUA_MoldTemperature,StartShotSN,EndShotSN,GroupSN,Type
0,1,09:23.8,11:49.8,t11,#2-4^TOY,1.06,14.14879,48.305779,0.021944,-0.011339,0,0.99,9.154309,45.064541,0.021747,-0.010132,0.01,1.04,7.08935,42.935417,0.024223,-0.010319,0.02,0.48,1.98,-0.010371,0.7,5.582084,32.348705,26.766621,0.034745,562.98,1.44,32.348705,66.638178,34.289473,-0.037038,898.21,68.007075,82.950629,14.943554,0.22,68.05774,65.09,141.785707,1821.66,141.854845,150.930355,0,150.930355,0.07,3.241238,-0.01,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,1,1,1,0
1,2,12:12.2,14:38.1,t11,#2-4^TOY,0.96,8.488392,46.251426,0.020756,-0.010049,0,1.12,10.489044,43.460442,0.025541,-0.010781,0.06,0.99,10.312204,46.70804,0.021196,-0.010413,0,0.54,1.98,-0.010136,0.73,5.426738,35.197341,29.770603,0.030567,644.65,1.43,35.197341,66.642614,31.445273,-0.041024,822.68,67.962057,82.828366,14.866309,0.23,68.01281,64.68,141.784266,1825.97,141.852433,150.900857,17.399743,168.3006,-0.16,2.790984,-0.06,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2,2,2,0
2,3,15:00.4,17:26.5,t11,#2-4^TOY,0.98,16.411448,53.499554,0.018318,-0.011088,0,1.02,7.885406,44.623369,0.020841,-0.010282,0.02,1.09,11.335504,46.196566,0.023595,-0.011182,0.02,0.43,1.98,-0.013864,0.27,5.880455,35.693914,29.813459,0.030523,650.36,1.43,35.693914,66.642759,30.948845,-0.035866,808.26,66.592006,67.200962,0.608956,1.12,66.592006,2.52,141.882993,1820.92,141.95093,150.975584,17.396936,168.37252,-0.04,8.876185,-0.02,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,3,3,3,0
3,4,17:48.9,20:14.9,t11,#2-4^TOY,1.09,10.686955,43.714665,0.024935,-0.010695,0.04,1.03,7.681343,42.801217,0.023598,-0.010294,0.01,1.02,11.518071,48.078887,0.020175,-0.010873,0,0.27,1.98,-0.01205

In [7]:
data2.columns = data2.iloc[0]
data2 = data2.iloc[2:, :]
data2 = data2.reset_index(drop=True)
data2.head()

,ShotSN,MoldClampedTime,MoldReleasedTime,MOID,ProdID,Sensor2_MaxPressure,Sensor2_SumStage1Pressure,Sensor2_MaxPressureTime,Sensor2_AvgStage1PressureSlope,Sensor2_AvgStage2PressureSlope,Sensor2_MoldReleasePressure,Sensor3_MaxPressure,Sensor3_SumStage1Pressure,Sensor3_MaxPressureTime,Sensor3_AvgStage1PressureSlope,Sensor3_AvgStage2PressureSlope,Sensor3_MoldReleasePressure,Sensor4_MaxPressure,Sensor4_SumStage1Pressure,Sensor4_MaxPressureTime,Sensor4_AvgStage1PressureSlope,Sensor4_AvgStage2PressureSlope,Sensor4_MoldReleasePressure,Sensor5_StartTemperature,Sensor5_MaxTemperature,Sensor5_AvgStage2TemperatureSlope,Sensor5_MoldReleaseTemperature,Sensor1_Stage1StartTime,Sensor1_Stage1EndTime,Sensor1_Stage1Period,Sensor1_AvgStage1PressureSlope,Sensor1_SumStage1Pressure,Sensor1_Stage1EndPressure,Sensor1_Stage2StartTime,Sensor1_Stage2EndTime,Sensor1_Stage2Period,Sensor1_AvgStage2PressureSlope,Sensor1_SumStage2Pressure,Sensor1_Stage3StartTime,Sensor1_Stage3EndTime,Sensor1_Stage3Period,Sensor1_MaxStage3Pressure,Sensor1_MaxStage3PressureTime,Sensor1_SumStage3Pressure,Sensor1_CycleTime,Sensor1_SumPressure,Soft_MoldClampedCycleTime,Soft_MoldBasedCycleTime,Soft_MoldShotInterval,Soft_MoldCompleteCycleTime,Soft_DiffMaxPressure,Soft_DiffIncreasePressureTime,Soft_DiffMoldReleasePressure,OPCUA_CycleTime,OPCUA_MoldClampingTime,OPCUA_MoldReleasingTime,OPCUA_InjectionTime,OPCUA_VPChangeOverDelayTime,OPCUA_HoldingTime,OPCUA_PlastificationTime,OPCUA_VPChangeOverPosition,OPCUA_CusionVolume,OPCUA_EndHoldingPosition,OPCUA_EndPlastificationPosition,OPCUA_Nozzle1Temperature,OPCUA_Nozzle2Temperature,OPCUA_Nozzle3Temperature,OPCUA_Nozzle4Temperature,OPCUA_Nozzle5Temperature,OPCUA_IMMOilTemperature,OPCUA_FallMaterialTemperature,OPCUA_MaxInjectionPressure,OPCUA_VPChangeOverPressure,OPCUA_SumInjectionPressure,OPCUA_SumHoldingPressure,OPCUA_MaxCavityNearGatePressure,OPCUA_MaxCavityFarGatePressure,OPCUA_SumCavityPressure,OPCUA_SumMoldClampingForce,OPCUA_MaxMoldClampingForce,OPCUA_MaxBackPressure,OPCUA_DelayPlastificationTime,OPCUA_InMoldCoolingTemperature,OPCUA_OutMoldCoolingTemperature,OPCUA_MoldCoolingVolume,OPCUA_MoldReleasingPosition2,OPCUA_MoldReleasingPosition3,OPCUA_MoldReleasingPosition4,OPCUA_MoldReleasingPosition5,OPCUA_ScrewSuckBackPosition,OPCUA_ScrewRPM,OPCUA_PartWeight,OPCUA_GateClosingTime,OPCUA_MeltTemperature,OPCUA_MoldTemperature,StartShotSN,EndShotSN,GroupSN,Type
0,2254,10:29.6,12:54.0,t11,#2-4^TOY,0.05,0.066954,30.226254,0.001323,-0.000456,0,0.14,0,0.000345,0,-0.000858,0.02,0.1,1.218391,34.640692,0.002887,-0.00095,0,0.14,1.93,-0.010172,0.67,4.920607,27.99354,23.072933,0.037273,495.32,1.41,27.99354,50.007882,22.014342,-0.05133,274.74,49.957207,65.47114,15.513933,0.29,49.957207,79.62,139.891092,1205.09,139.959308,149.358556,17.110219,166.468775,-0.09,30.225909,-0.02,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2254,2254,2254,1
1,2255,13:16.0,15:40.4,t11,#2-4^TOY,0.11,1.101516,27.546706,0.003993,-0.000979,0,0.03,0.49645,44.776663,0.000447,-0.000105,0.02,0.03,0,0.000369,0,-0.000214,0,0.52,2.02,-0.010048,0.77,5.429687,27.496024,22.066337,0.038973,473.77,1.41,27.496024,50.048128,22.552104,-0.050106,269.82,49.997431,65.512633,15.515202,0.28,49.997431,79.42,139.879328,1182.87,139.949203,149.293589,17.113158,166.406747,0.08,-17.229957,-0.02,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2255,2255,2255,1
2,2256,16:02.5,18:26.4,t11,#2-4^TOY,0.05,0.544567,57.047626,0.000701,-0.000603,0,0.03,0.332434,30.836741,0.000324,-9.20E-05,0.02,0.19,3.990315,29.822788,0.006371,-0.001453,0.03,0.32,2.02,-0.010208,0.75,4.916987,27.495589,22.578602,0.038089,481.51,1.41,27.495589,50.018569,22.52298,-0.050171,273.49,49.9679,65.536865,15.568965,0.29,49.9679,78.73,139.902714,1190.77,139.974908,148.881964,17.084984,165.966948,0.02,26.210885,-0.02,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2256,2256,2256,1
3,2257,18:48.8,21:13.2,t11,#2-4^TOY,0.14,2.031487,35.601051,0.003933,-0.000959,0.04,0.02,0.603321,60.307064,0.000166,-0.000126,0.01,0.13,0,0.000366,0,-0.000858,0.01,0.29,1.99,-0.009805,

In [8]:
data_all = pd.concat([data1, data2])
data_all.shape

(4690, 99)

In [9]:
df = data_all[all_name]
df.shape

(4690, 31)

In [10]:
df = df.apply(pd.to_numeric, errors='coerce')

In [11]:
df_nona = df.dropna()
df_nona.shape

(4655, 31)

In [12]:
X = df_nona.iloc[:, :-1]
y = df_nona['Type']
X.shape, y.shape

((4655, 30), (4655,))

In [13]:
y[y==1] = -1  # 将异常点标记为-1
y[y==0] = 1   # 将正常点标记为1

In [14]:
y.unique()

array([ 1, -1])

In [18]:

# 生成二元分类数据集，10%的样本标记为异常点
# X, y = make_classification(n_samples=100, n_features=20, n_informative=10, n_redundant=0, n_classes=2, weights=[0.9, 0.1], random_state=42)
# y[y==1] = -1  # 将异常点标记为-1
# y[y==0] = 1   # 将正常点标记为1

# 划分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 创建One-Class SVM模型并拟合训练数据
ocsvm = OneClassSVM(kernel='rbf', nu=0.1)
ocsvm.fit(X_train)

# 使用测试集评估模型性能
y_pred = ocsvm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label=-1)
recall = recall_score(y_test, y_pred, pos_label=-1)
f1 = f1_score(y_test, y_pred, pos_label=-1)

print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1: {f1:.3f}")
print(f'MAE: {mae(y_test, y_pred)}')
print(f'MSE: {mse(y_test, y_pred)}')
print(f'RMSE: {sqrt(mse(y_test, y_pred))}')
print(f'R2: {r2_score(y_test, y_pred)}')

# # 使用模型预测新的样本是否为异常点
# new_sample = [[0.5, 0.3, -0.2, 1.0, -1.2, 0.8, -0.1, -1.5, 0.7, -0.9, 0.3, 0.2, 0.1, 0.6, -1.0, 0.9, -0.7, 0.5, -1.1, -0.3]]

# if ocsvm.predict(new_sample) == 1:
#     print("This is a normal sample.")
# else:
#     print("This is an anomalous sample.")


Accuracy: 0.905
Precision: 0.178
Recall: 0.783
F1: 0.290
MAE: 0.18904403866809882
MSE: 0.37808807733619765
RMSE: 0.6148886706845376
R2: -2.923003256081212


In [22]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 18,   5],
       [ 83, 825]])

In [24]:
'''
            预测为正类   预测为负类
实际为正类       TP        FN
实际为负类       FP        TN

'''

'\n            预测为正类   预测为负类\n实际为正类       TP        FN\n实际为负类       FP        TN\n\n'

In [23]:
list(zip(y_test, y_pred))

[(1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (-1, -1),
 (1, 1),
 (1, 1),
 (1, -1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, -1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, -1),
 (1, 1),
 (-1, 1),
 (1, 1),
 (1, 1),
 (1, -1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, -1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, -1),
 (1, 1),
 (1, -1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
